<a href="https://colab.research.google.com/github/spark-ming/albert-qa-demo/blob/master/Question_Answering_with_ALBERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Reading Comprehension with ALBERT (and similar)

Author: [@techno246](https://twitter.com/techno246)

## Introduction

Reading comprehension, otherwise known as Question Answer systems, are one of the tasks that NLP tries to solve. The goal of this task is to be able to answer an arbitary question given a context. For instance, given the following context:

> New Zealand (Māori: Aotearoa) is a sovereign island country in the southwestern Pacific Ocean. The country has two main landmasses — the North Island (Te Ika-a-Māui), and the South Island (Te Waipounamu) — and around 600 smaller islands. It has a total land area of 268,000 square kilometres (103,500 sq mi), and a population of 4.9 million.  Because of its remoteness, it was one of the last lands to be settled by humans. During its long period of isolation, New Zealand developed a distinct biodiversity of animal, fungal, and plant life. The country's varied topography and its sharp mountain peaks, such as the Southern Alps, owe much to the tectonic uplift of land and volcanic eruptions. New Zealand's capital city is Wellington, and its most populous city is Auckland.

We ask the question

> What is the capital of New Zealand? 

We expect the QA system is to respond with something like this:

> Wellington

Since 2017, transformer models have shown to outperform existing approaches for this task. Many variations of transformer models exist, including BERT, RoBERTA, XLNET. One of the newcomers to the group is ALBERT (A Lite BERT) which was published in September 2019. The research group claims that it outperforms BERT, with much less parameters (shorter training and inference time). 

This tutorial demonstrates how you can fine-tune a transformer model for the task of QnA and use it for inference. For this tutorial, we will use the transformer library built by Hugging Face, which is an extremely nice implementation of the transformer models (including ALBERT) in both TensorFlow and PyTorch. You can also just download a fine tuned model [here](https://huggingface.co/models) (which I encourage in general to save money and reduce emissions), however for educational purposes we will train a small version ourselves. 

Note that the goal of this is not to build an optimised, production ready system, but to demonstrate the concept with as little code as possible. Therefore a lot of code will be retrofitted for this purpose. 

## 1.0 Setup

If you're going to train your own model, you're going to need som RAM. If not, skip this step

Here's a hack: you can crash our notebook instance to get some more RAM (25GB instead of default 12GB). Trust me, it works. When the runtime crashes, it will ask you if you want more RAM. Just press yes.

Note **don't run this a second time**.

In [4]:
# Colab Hack
a = ['crash me']
while(1):
    a = a + a

KeyboardInterrupt: ignored

Let's check out what kind of GPU our friends at Google gave us. Hope it's a P100!

In [1]:
!nvidia-smi

Mon Jan 20 11:53:27 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.44       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

First, we copy over the Github.


Note it's checking out a specific commit only because I've tested this

In [0]:
!git clone https://github.com/huggingface/transformers \
&& cd transformers \
&& git checkout a3085020ed0d81d4903c50967687192e3101e770 

In [0]:
!pip install ./transformers
!pip install tensorboardX --user

## 2.0 Train Model

This is where we can train our own model. Note you can skip this step if you don't want to wait 1.5 hours!

### 2.1 Get Training and Evaluation Data

The SQuAD dataset contains question/answer pairs to for training the ALBERT model for the QA task. 

Now get the SQuAD V2.0 dataset. `train-v2.0.json` is for training and `dev-v2.0.json` is for evaluation to see how well your model trained.

Read more about this dataset here: https://rajpurkar.github.io/SQuAD-explorer/

[Albert Learns to Read](https://littlealbert.now.sh) is trained on SQuAD V2.0. I'm using V1.1 in this demo because it keeps crashing the Colab runtime (runs out of memory). To train against 2.0 on your own machine, just change "1.1" to "2.0"

In [7]:
!mkdir dataset \
&& cd dataset \
&& wget https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v2.0.json \
&& wget https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v2.0.json

--2020-01-20 11:08:54--  https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v2.0.json
Resolving rajpurkar.github.io (rajpurkar.github.io)... 185.199.109.153, 185.199.108.153, 185.199.110.153, ...
Connecting to rajpurkar.github.io (rajpurkar.github.io)|185.199.109.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 42123633 (40M) [application/json]
Saving to: ‘train-v2.0.json’

train-v2.0.json     100%[===================>]  40.17M  24.8MB/s    in 1.6s    

2020-01-20 11:08:58 (24.8 MB/s) - ‘train-v2.0.json’ saved [42123633/42123633]

--2020-01-20 11:08:58--  https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v2.0.json
Resolving rajpurkar.github.io (rajpurkar.github.io)... 185.199.109.153, 185.199.108.153, 185.199.110.153, ...
Connecting to rajpurkar.github.io (rajpurkar.github.io)|185.199.109.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4370528 (4.2M) [application/json]
Saving to: ‘dev-v2.0.json’

dev-v2.0.json       

### 2.2 Run training 

We can now train the model with the training set. 

### Notes about parameters:
`per_gpu_train_batch_size` specifies the number of training examples per iteration per GPU. *In general*, higher means more accuracy and faster training. However, the biggest limitation is the size of the GPU. 12 is what I use for a GPU with 16GB memory. 

`save_steps` specifies number of steps before it outputs a checkpoint file. I've increased it to save disk space.

`num_train_epochs` I recommend two epochs here. It's currently set to one for the purpose of time

Warning: it takes about 1.5 hours to train an epoch! If you don't want to wait this long, feel free to skip this step and note the comment in the code to use a pretrained model!

In [8]:
!export SQUAD_DIR=/content/dataset \
&& python transformers/examples/run_squad.py \
  --model_type albert \
  --model_name_or_path albert-base-v2 \
  --do_train \
  --do_eval \
  --do_lower_case \
  --train_file $SQUAD_DIR/train-v2.0.json \
  --predict_file $SQUAD_DIR/dev-v2.0.json \
  --per_gpu_train_batch_size 12 \
  --learning_rate 3e-5 \
  --num_train_epochs 1.0 \
  --max_seq_length 384 \
  --doc_stride 128 \
  --output_dir /content/model_output \
  --save_steps 1000 \
  --threads 4 \
  --version_2_with_negative 

01/20/2020 11:09:08 - WARNING - __main__ -   Process rank: -1, device: cuda, n_gpu: 1, distributed training: False, 16-bits training: False
01/20/2020 11:09:08 - INFO - filelock -   Lock 139840643170256 acquired on /root/.cache/torch/transformers/0bbb1531ce82f042a813219ffeed7a1fa1f44cd8f78a652c47fc5311e0d40231.ea38adf566af403b92e06f53ac1b443ebd324162fc74a90de95609963d429505.lock
01/20/2020 11:09:08 - INFO - transformers.file_utils -   https://s3.amazonaws.com/models.huggingface.co/bert/albert-base-v2-config.json not found in cache or force_download set to True, downloading to /root/.cache/torch/transformers/tmpcnlc359m
Downloading: 100% 483/483 [00:00<00:00, 443kB/s]
01/20/2020 11:09:09 - INFO - transformers.file_utils -   storing https://s3.amazonaws.com/models.huggingface.co/bert/albert-base-v2-config.json in cache at /root/.cache/torch/transformers/0bbb1531ce82f042a813219ffeed7a1fa1f44cd8f78a652c47fc5311e0d40231.ea38adf566af403b92e06f53ac1b443ebd324162fc74a90de95609963d429505
01/20/

## 3.0 Setup prediction code

Now we can use the Hugging Face library to make predictions using our newly trained model. Note that a lot of the code is pulled from `run_squad.py` in the Hugging Face repository, with all the training parts removed. This modified code allows to run predictions we pass in directly as strings, rather .json format like the training/test set.

NOTE if you decided train your own mode, change the flag `use_own_model` to `True`


In [11]:
import os
import torch
import time
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

from transformers import (
    AlbertConfig,
    AlbertForQuestionAnswering,
    AlbertTokenizer,
    squad_convert_examples_to_features
)

from transformers.data.processors.squad import SquadResult, SquadV2Processor, SquadExample

from transformers.data.metrics.squad_metrics import compute_predictions_logits

# READER NOTE: Set this flag to use own model, or use pretrained model in the Hugging Face repository
use_own_model = False

if use_own_model:
  model_name_or_path = "/content/model_output"
else:
  model_name_or_path = "ktrapeznikov/albert-xlarge-v2-squad-v2"

output_dir = ""

# Config
n_best_size = 1
max_answer_length = 30
do_lower_case = True
null_score_diff_threshold = 0.0

def to_list(tensor):
    return tensor.detach().cpu().tolist()

# Setup model
config_class, model_class, tokenizer_class = (
    AlbertConfig, AlbertForQuestionAnswering, AlbertTokenizer)
config = config_class.from_pretrained(model_name_or_path)
tokenizer = tokenizer_class.from_pretrained(
    model_name_or_path, do_lower_case=True)
model = model_class.from_pretrained(model_name_or_path, config=config)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model.to(device)

processor = SquadV2Processor()

def run_prediction(question_texts, context_text):
    """Setup function to compute predictions"""
    examples = []

    for i, question_text in enumerate(question_texts):
        example = SquadExample(
            qas_id=str(i),
            question_text=question_text,
            context_text=context_text,
            answer_text=None,
            start_position_character=None,
            title="Predict",
            is_impossible=False,
            answers=None,
        )

        examples.append(example)

    features, dataset = squad_convert_examples_to_features(
        examples=examples,
        tokenizer=tokenizer,
        max_seq_length=384,
        doc_stride=128,
        max_query_length=64,
        is_training=False,
        return_dataset="pt",
        threads=1,
    )

    eval_sampler = SequentialSampler(dataset)
    eval_dataloader = DataLoader(dataset, sampler=eval_sampler, batch_size=10)

    all_results = []

    for batch in eval_dataloader:
        model.eval()
        batch = tuple(t.to(device) for t in batch)

        with torch.no_grad():
            inputs = {
                "input_ids": batch[0],
                "attention_mask": batch[1],
                "token_type_ids": batch[2],
            }

            example_indices = batch[3]

            outputs = model(**inputs)

            for i, example_index in enumerate(example_indices):
                eval_feature = features[example_index.item()]
                unique_id = int(eval_feature.unique_id)

                output = [to_list(output[i]) for output in outputs]

                start_logits, end_logits = output
                result = SquadResult(unique_id, start_logits, end_logits)
                all_results.append(result)

    output_prediction_file = "predictions.json"
    output_nbest_file = "nbest_predictions.json"
    output_null_log_odds_file = "null_predictions.json"

    predictions = compute_predictions_logits(
        examples,
        features,
        all_results,
        n_best_size,
        max_answer_length,
        do_lower_case,
        output_prediction_file,
        output_nbest_file,
        output_null_log_odds_file,
        False,  # verbose_logging
        True,  # version_2_with_negative
        null_score_diff_threshold,
        tokenizer,
    )

    return predictions

## 4.0 Run predictions

Now for the fun part... testing out your model on different inputs. Pretty rudimentary example here. But the possibilities are endless with this function.

In [12]:
context = "My favourite colour is blue"
questions = ["What's your favourite colour?"]

# Run method
predictions = run_prediction(questions, context)

# Print results
for key in predictions.keys():
  print(predictions[key])

add example index and unique id: 100%|██████████| 1/1 [00:00<00:00, 3844.46it/s]


blue


## 5.0 Next Steps

In this tutorial, you learnt how to fine-tune an ALBERT model for the task of question answering, using the SQuAD dataset. Then, you learnt how you can make predictions using the model. 

We retrofitted `compute_predictions_logits` to make the prediction for the purpose of simplicity and minimising dependencies in the tutorial. Take a peak inside that module to see how it works. If you want to serve this as an API, you will want to strip out a lot of the stuff it's doing (such as writing the predictions to a JSON, etc)

You can now turn this into an API by serving it using a web framework. I recommend checking out FastAPI, which is what [Albert Learns to Read](https://littlealbert.now.sh) is built on. 

Feel free to tweet me on @techno246 if you have any questions!